In [1]:
from __future__ import division
import numpy as np
import scipy.linalg as slg
import math
import sys
from numpy import genfromtxt
import csv
from numpy import linalg as LA
from numpy.linalg import inv

In [14]:
def str2float(s):
    return float(s.replace('D','e'))

def CutLTMatOut(fname, head_line, index=0):
    cur_index = 0
    f = open(fname)
    status = None
    r = 0
    indices = []
    mat = []
    for line in f:
        if head_line in line:
            status = "Column Indices"
            continue
        if status == "Rows":
            splitted = line.split()
            if all([x.isnumeric() for x in splitted]):
                status = "Column Indices"
            else:
                try:
                    r = int(splitted[0])
                    floats = [str2float(s) for s in splitted[1:]]
                    if len(mat) < r:
                        mat.append(floats)
                    else:
                        mat[r-1] += floats
                except ValueError as e:
                    break
                continue
        if status == "Column Indices":
            status = "Rows"
            indices = [int(I) for I in line.split()]
            continue
    f.close()
    return mat

def toNPMatrix(mat_array, symmetry = 'symmetric'):
    dim = len(mat_array)
    mat = np.zeros((dim,dim))
    for i in range(dim):
        mat[i,:i+1] = mat_array[i]
    if symmetry == 'symmetric':
        diag = np.diag(np.diag(mat))
        return mat + mat.T - diag
    elif symmetry == 'anti-symmetric':
        return mat - mat.T


def ReadMatrix(fname, head_line, index=0):
    return np.array(CutLTMatOut(fname, head_line, index))

def ReadLowerTri(fname, head_line, index=0, symmetry = 'symmetric'):
    return toNPMatrix(CutLTMatOut(fname, head_line, index), symmetry)

def outMatrix(A):
    if np.iscomplexobj(A):
        print("Real part:")
        outMatrix(A.real)
        print()
        print("Imag part:")
        outMatrix(A.imag)
        return
    print("--------------------------------------------------------------------------------")
    curColStart, curColEnd = 0, 5
    while curColStart < A.shape[1]:
        curColEnd = min(curColStart + 5, A.shape[1])
        print("     "+"               ".join(list(map(str, range(curColStart + 1, curColEnd + 1)))))
        for i in range(A.shape[0]):
            print("%3d   "%(i+1,) + " ".join(list(map(lambda x : "%15.8e"%(x if abs(x) > 1e-8 else 0,) , A[i, curColStart : curColEnd]))))
        curColStart += 5
        print()

    print("--------------------------------------------------------------------------------")

def RMSD(V, W):
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = np.array(V) - np.array(W)
    N = len(V)
    return np.sqrt( np.trace(diff * diff) / N)
    

In [9]:
fname = "./neo_small_basis_stw.log"
last_Den_hl = 'Newest Densitylol3'
cur_Den_hl = 'Newest Densitylol4'


In [10]:
last_Den = ReadLowerTri(fname, last_Den_hl)
cur_Den = ReadLowerTri(fname, cur_Den_hl)

In [11]:
outMatrix(last_Den)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    9.41329000e-01  2.12384000e-01  0.00000000e+00 -1.00607000e-01  0.00000000e+00
  2    2.12384000e-01  4.79184000e-02  0.00000000e+00 -2.26992000e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4   -1.00607000e-01 -2.26992000e-02  0.00000000e+00  1.07527000e-02  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  9.41329000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.12384000e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00607000e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [12]:
outMatrix(cur_Den)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    9.41329000e-01  2.12387000e-01  0.00000000e+00 -1.00603000e-01  0.00000000e+00
  2    2.12387000e-01  4.79196000e-02  0.00000000e+00 -2.26985000e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4   -1.00603000e-01 -2.26985000e-02  0.00000000e+00  1.07518000e-02  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  9.41329000e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.12387000e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00603000e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [15]:
RMSD(last_Den,cur_Den)

7.499999999998827e-07